**Run this cell first to provision a cloud machine and set up Drake.**

This may take a minute or two the first time, but once a machine is set up it should be good for about 12 hours. 

In [1]:
#@title Run Initialization
import importlib
import sys
import os
from urllib.request import urlretrieve
import subprocess
import shutil

assert 'google.colab' in sys.modules, "This notebook is meant to be run in google colab!"

drake_url = "https://drake-packages.csail.mit.edu/tmp/drake-0.27.0-pip-bionic.tar.gz"
if importlib.util.find_spec('pydrake') is None:
    # We're in colab and don't have pydrake, so install it on the cloud machine.
    if os.path.isdir('/opt/drake'):
        shutil.rmtree('/opt/drake')
    print("Installing Drake")
    urlretrieve(drake_url, 'drake.tar.gz')
    subprocess.run(['mkdir', '/opt/drake'])
    subprocess.run(['tar', '-xzf', 'drake.tar.gz', '-C', '/opt/drake'], check=True)
    
    print("Installing other dependencies")
    subprocess.run(["pip3", "install", "meshcat"])
    subprocess.run(['apt-get', 'update', '-o', 'APT::Acquire::Retries=4', '-qq'], check=True)
    with open("/opt/drake/share/drake/setup/packages-bionic.txt", "r") as f:
        packages = f.read().splitlines()
    subprocess.run(['apt-get', 'install', '-o',
                    'APT::Acquire::Retries=4', '-o', 'Dpkg::Use-Pty=0',
                    '-qy', '--no-install-recommends'] + packages,
                    check=True)
    
    v = sys.version_info
    path = f"/opt/drake/lib/python{v.major}.{v.minor}/site-packages"
    if importlib.util.find_spec('pydrake') is None:
        sys.path.append(path)

# Start a meshcat server
print("Starting Meshcat")
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=['--ngrok_http_tunnel'])

# Clone our github repo
install_path = '/opt/passivity_cbf_demo'
if not os.path.isdir(install_path):
    print("Cloning github repo")
    subprocess.run(['git','clone','https://github.com/vincekurtz/passivity_cbf_demo.git',install_path])
sys.path.append(install_path)

print("Done!")


Installing Drake
Installing other dependencies
Starting Meshcat
Cloning github repo
Done!


While you're waiting, here's what this is all about...

Run the following cell to choose what control method to use and what type of constraints to apply, if any.

In [2]:
#@title Set Simulation Parameters
import ipywidgets as widgets

ctrl_radio = widgets.RadioButtons(
    description="Control Method:",
    style={'description_width':'initial'},
    options=["Unconstrained","Standard","Passivity Guaranteed"],
    value="Passivity Guaranteed")
cons_radio = widgets.RadioButtons(
    description="Constraint Type:",
    style={'description_width':'initial'},
    options=["Singularity Avoidance", "Joint Limits", "None"],
    value="Singularity Avoidance")

display(ctrl_radio)
display(cons_radio)

RadioButtons(description='Control Method:', index=2, options=('Unconstrained', 'Standard', 'Passivity Guarante…

RadioButtons(description='Constraint Type:', options=('Singularity Avoidance', 'Joint Limits', 'None'), style=…

Run the following cell to initialize the simulation with the options specified above.

In [3]:
#@title Initialize Simulation
from colab_utils import *

simulator, gui = setup_colab_simulation(ctrl_radio.value, cons_radio.value, 
                                   install_path, zmq_url, include_manipuland=True)


setting up display
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://ebd9ee43dab3.ngrok.io/static/
Connected to meshcat-server.
Ready to Simulate!


/opt/drake/lib/python3.7/site-packages/pydrake/systems/meshcat_visualizer.py:417: UserWarning: Unsupported shape <pydrake.geometry.HalfSpace object at 0x7ff950eb41f0> ignored
  warnings.warn(f"Unsupported shape {shape} ignored")


You can open the URL above to see the simulation in a different window, or run the block below to embed the visualizer in this page. 






In [4]:
#@title Show Visualizer
import IPython
width=800
height=400
if web_url[:5] != 'https':
    web_url = 'https' + web_url[4:]
iframe = '<iframe src=' + web_url + ' width=' + str(width) + ' height=' + str(height) + '></iframe>'
IPython.display.HTML(iframe)

In [5]:
#@title Start Simulating!
import threading

# Show sliders for controlling the target end-effector pose
display_gui(gui)
stop_button = widgets.ToggleButton(value=False, description='Stop Simulation')
display(stop_button)

# Run the simulation in a separate thread so we can stop it with a button press
def run_sim():
    while not stop_button.value:
        simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)
thread = threading.Thread(target=run_sim)
thread.start()


FloatSlider(value=3.141592653589793, description='Roll', max=6.283185307179586, min=-6.283185307179586, step=0…

FloatSlider(value=0.0, description='Pitch', max=1.2707963267948965, min=-1.2707963267948965, step=0.01)

FloatSlider(value=1.5707963267948966, description='Yaw', max=6.283185307179586, min=-6.283185307179586, step=0…

FloatSlider(value=0.2, description='X', max=1.0, min=-1.0, step=0.01)

FloatSlider(value=0.4, description='Y', max=1.0, min=-1.0, step=0.01)

FloatSlider(value=0.5, description='Z', max=1.0, step=0.01)

ToggleButton(value=False, description='Toggle Gripper')

ToggleButton(value=False, description='Stop Simulation')